In [ ]:
import copy
def make_mode(source_vocab,target_vocab,N = 6,d_model=512,d_ff=2048,head = 8,dropout = 0.1):
    """该函数用来构建模型, 有7个参数，分别是源数据特征(词汇)总数，目标数据特征(词汇)总数，
       编码器和解码器堆叠数，词向量映射维度，前馈全连接网络中变换矩阵的维度，
       多头注意力结构中的多头数，以及置零比率dropout."""
    c = copy.deepcopy
    #多头注意力机制
    attn = MultiHeadAttention(head,d_model,dropout)
    
    #位置编码
    position = PositionalEncoding(d_model,dropout)
    
    #前馈连接层
    ff = PositionwiseFeedForward(d_model,d_ff,dropout)
    
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model,c(attn),c(ff),dropout),N),
        Decoder(DecoderLayer(d_model,c(attn),c(attn),c(ff),dropout),N),
        nn.Sequential(Embeddings(d_model, source_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, target_vocab), c(position)),
        Generator(d_model,target_vocab)
    )
    
     # 模型结构完成后，接下来就是初始化模型中的参数，比如线性层中的变换矩阵
    # 这里一但判断参数的维度大于1，则会将其初始化成一个服从均匀分布的矩阵，
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model
    